In [18]:
"""
Takes raw exports from Realsense export tool and Cepton pcap conversion tool
Renames files and matches them based on timestamp. Does not modify source directory
Inputs:
    <date>/
        <camerafile>.bag
        <lidarfile>.pcap
        <lidarfile>.json
        <gpsfile>.ubx
            
Outputs:
    Export/
        image_00/
            Data/
                0000000001.png
                ...
            timestamps.txt
        pointcloud/
            Data/
                0000000001.bin
                ...
            timestamps.txt

Calibration and GPS data are currently handled separately

Command to start PTP timestamping on interface enp118s0 (Eth0)
sudo ptp4l -S -i enp118s0 -m
"""

import shutil
import pandas as pd
import numpy as np
import os
import open3d as o3d
from enum import Enum
from os import listdir
from os.path import isfile, join
from pyubx2 import ERR_IGNORE, GET, POLL, SET, UBXReader, UBX_PROTOCOL, NMEA_PROTOCOL, RTCM3_PROTOCOL
from ouster.sdk import open_source
from ouster.sdk import client
from ouster.sdk import pcap
from tkinter import *
from tkinter.filedialog import askdirectory

class FileTypes(Enum):
    PCD = 0
    BIN = 1

class IMUTypes(Enum):
    REALSENSE_IMU = 0
    OUSTER_IMU = 1
 
    
#
#Options
#
OUTPUT_TYPE = FileTypes.BIN
IMU = IMUTypes.OUSTER_IMU

# Promt user for raw data directory
rawdatapath = askdirectory(title='Select raw data directory')
dirfiles = listdir(rawdatapath)
path = f"{rawdatapath}/"

# Parse date and paths
parts = rawdatapath.split('/')
date = pd.Timestamp(parts[-1])
cam_frame_path = path + "Camera/"

# Required files
lidar_pcap = ''
lidar_json = ''
gps_file = ''
cam_bag = ''
imu_file = ''

# Find required files
for file in dirfiles:
    if(file.endswith('.pcap')):
       lidar_pcap = path + file
    elif(file.endswith('.json')):
        lidar_json = path + file
    elif(file.endswith('.ubx') or file.endswith('.log')):
        gps_file = path + file
    elif(file.endswith('.bag')):
        cam_bag = path + file

# Error checking for required data
if(lidar_pcap == ''):
    raise Exception('ERROR: Lidar pcap not found!')
if(lidar_json == ''):
    raise Exception('ERROR: Lidar json not found!')
if(gps_file == ''):
    #raise Exception('ERROR: GPS file not found!')
    pass
if(os.path.exists(cam_frame_path) == False and cam_bag == ''):
    raise Exception('No camera data found!')

# Destination paths
dest_path = path + "Export/"
cam_dest = dest_path + 'image_00/'
lidar_dest = dest_path + 'pointcloud/'
gps_dest = dest_path + 'navigation/'

imu_file = ''

#Use realsense sdk to convert .bag to frames if none exist yet
if(os.path.exists(cam_frame_path) == False):
    print('Camera frames not found, converting from bag...')
    os.mkdir(cam_frame_path)
    os.system(f'rs-convert -i {cam_bag} -p {cam_frame_path}')
    print('Exporting IMU data...')
    os.system(f'rs-convert -i {cam_bag} -v {cam_frame_path}')

# Reference to all files in the camera folder
cam_files = [f for f in listdir(cam_frame_path) if isfile(join(cam_frame_path, f))]
cam_files.sort()

# Find IMU file
for file_name in cam_files:
    if('imu_pose' in file_name):
        imu_file = file_name
        break

if (imu_file == ''):
    print('Exporting IMU data...')
    os.system(f'rs-convert -i {cam_bag} -v {cam_frame_path}')

    # Find IMU file
    for file_name in cam_files:
        if('imu_pose' in file_name):
            imu_file = file_name
            break

if(imu_file == ''):
    raise Exception("Error: IMU Data not found!")

print("All files found successfully")

All files found successfully


In [19]:
"""
Create dataframe of raw camera frame metadata from frame filename. [Epoch, Datetime, Filepath]
"""

# Camera metadata dataframe
cam_dataframe = pd.DataFrame()
temp_data = []

# Iterate through each file in the camera frame files
for file_name in cam_files:

    # If the file is not metadata
    if ('metadata' not in file_name) and ('imu_pose' not in file_name):
        # Get epoch from filename
        trimmedname = file_name.split('_')
        parts = trimmedname[2].split('.')

        # Append data to frame
        temp_dict = { 'Epoch' : float(parts[0] + '.' + parts[1]), 
                     'Datetime' : pd.to_datetime(float(parts[0] + '.' + parts[1]),unit='ms'),
                     'camera_path' : cam_frame_path + file_name }
        
        # Append camera frame info to running list
        temp_data.append(temp_dict)

# Keep list of new camera frames
#cam_data['Datetime'] = pd.to_datetime(cam_data['Epoch'])
cam_dataframe = pd.DataFrame(temp_data)
cam_dataframe['Datetime'] = cam_dataframe['Datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Los_Angeles')
print(cam_dataframe.head())

          Epoch                            Datetime  \
0  1.725935e+12 2024-09-09 19:21:43.678851074-07:00   
1  1.725935e+12 2024-09-09 19:21:43.712199707-07:00   
2  1.725935e+12 2024-09-09 19:21:43.745549316-07:00   
3  1.725935e+12 2024-09-09 19:21:43.778897949-07:00   
4  1.725935e+12 2024-09-09 19:21:43.812247559-07:00   

                                         camera_path  
0  /home/khanj/Dev/PerceptionTools/DriveData/2024...  
1  /home/khanj/Dev/PerceptionTools/DriveData/2024...  
2  /home/khanj/Dev/PerceptionTools/DriveData/2024...  
3  /home/khanj/Dev/PerceptionTools/DriveData/2024...  
4  /home/khanj/Dev/PerceptionTools/DriveData/2024...  


In [20]:
"""
Parse metadata from ouster pcap into pandas dataframe
1. index frames from pcap file
2. record frame time start and end
3. destagger point representation
4. convert destaggered frame points and reflectivity to binary as float32 types
"""

# Empty lists of desired data to be combined later
frame_ids = []
timestarts = []
timeends = []
xyzr = []
filepaths = []

imu_accel = []
imu_gyro  = []

# Load .pcap and .json files
print(lidar_pcap)
source = open_source(lidar_pcap, meta=[lidar_json], index = True)
lidar_metadata = source.metadata
print('frame count: ' + str(len(source)))

# Create binary data directory if doesnt exist
if(os.path.exists(f'{path}pointcloud/') == False):
    os.mkdir(f'{path}pointcloud/')
if(os.path.exists(f'{path}pointcloud/Data/') == False):
    os.mkdir(f'{path}pointcloud/Data/')
    print('Saved point cloud not found. Regenerating...')

# Get function from factory to project scan data to cartesian coordinates
xyzl = client.XYZLut(lidar_metadata)
packet_format = client.PacketFormat(lidar_metadata)

# Iterate through all lidar scan frames
print('Scanning frames...')
for scan in source:
    # If the scan is valid
    if scan:
        # Grab frame id
        frame_ids.append(scan.frame_id)
        # Grab time start and end for frame
        timestarts.append(scan.timestamp[0])
        timeends.append(scan.timestamp[-1]) 
        
        # Project points to cartesian and destagger representation
        pos = client.destagger(lidar_metadata,xyzl(scan))

        # Destagger reflectivity
        ref = client.destagger(lidar_metadata,scan.field(client.ChanField.REFLECTIVITY))

        # Reshape reflectivity to match XZY's shape
        reflectivity_reshaped = np.expand_dims(ref, axis=2)

        # Concatenate XYZ and reflectivity and cast to 32-bit floats
        xyzref_array = np.concatenate((pos, reflectivity_reshaped), axis=2).astype(np.float32)
        xyzr.append(xyzref_array)

        if(OUTPUT_TYPE == FileTypes.BIN):
            extension = "bin"
            
        elif(OUTPUT_TYPE == FileTypes.PCD):
            extension = "pcd"

        # Grab filepath of point representation
        filepath = f"{path}pointcloud/Data/{scan.frame_id:010d}." + extension    
        filepaths.append(filepath)  

        # Write frame data to disk if doesn't exist
        if(os.path.exists(filepath) == False):

            # Output files as binary
            if(OUTPUT_TYPE == FileTypes.BIN):
                # Get byte representation of XYZR
                scanbytes = bytearray(xyzref_array)

                # Write byte representation to pointcloud/Data/ directory
                file = open(filepath, 'wb')
                file.write(scanbytes)    

            # Output files as PCD
            elif(OUTPUT_TYPE == FileTypes.PCD):
                # Reshape array
                xyzref_array = xyzref_array.reshape((-1,4))[:,0:3]

                # Convert to Open3D point cloud
                o3d_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(xyzref_array))

                print(f'writing to: {filepath}')
                # Save to whatever format you like
                o3d.io.write_point_cloud(filepath, o3d_pcd)

            else:
                print(f'{scan.frame_id} already exists, skipping...')

print('Building lidar dataframe')
# Build pandas dataframe of desired lidar scan data
lidar_dataframe = pd.DataFrame()
lidar_dataframe['lidar_id'] = pd.DataFrame(frame_ids)
lidar_dataframe['Timestart'] = pd.DataFrame(timestarts)
lidar_dataframe['Timeend'] = pd.DataFrame(timeends)
lidar_dataframe['lidar_path'] = pd.DataFrame(filepaths)

lidar_dataframe.to_csv('timetest.csv')

# Convert time strings to datetime objects
lidar_dataframe['Timestart'] = pd.to_datetime(lidar_dataframe['Timestart'],unit='ns')
lidar_dataframe['Timeend'] = pd.to_datetime(lidar_dataframe['Timeend'],unit='ns')

# Convert from GMT to PST timezone
lidar_dataframe['Timestart'] = lidar_dataframe['Timestart'].dt.tz_localize('GMT').dt.tz_convert('America/Los_Angeles')
lidar_dataframe['Timeend'] = lidar_dataframe['Timeend'].dt.tz_localize('GMT').dt.tz_convert('America/Los_Angeles')

# Assign scan midpoint as frame epoch   
lidar_dataframe['Datetime'] = pd.DataFrame(lidar_dataframe['Timestart'] + ((lidar_dataframe['Timeend']-lidar_dataframe['Timestart'])/2))
print(lidar_dataframe.head())

/home/khanj/Dev/PerceptionTools/DriveData/2024-09-09/2024099_1921_OS-2-128_122414001752.pcap
loading metadata from ['/home/khanj/Dev/PerceptionTools/DriveData/2024-09-09/2024099_1921_OS-2-128_122414001752.json']
[2024-09-23 10:33:46.649] [ouster::sensor] [info] parsing non-legacy metadata format
 [####################] 100.0% indexed
finished building index
frame count: 4527
Scanning frames...
Building lidar dataframe
   lidar_id                           Timestart  \
0      2224           1969-12-31 16:00:00-08:00   
1      2225 2024-09-09 19:21:49.466755093-07:00   
2      2226 2024-09-09 19:21:49.566716380-07:00   
3      2227 2024-09-09 19:21:49.666685136-07:00   
4      2228 2024-09-09 19:21:49.766677044-07:00   

                              Timeend  \
0 2024-09-09 19:21:49.466666149-07:00   
1 2024-09-09 19:21:49.566618911-07:00   
2 2024-09-09 19:21:49.666578590-07:00   
3 2024-09-09 19:21:49.766573072-07:00   
4 2024-09-09 19:21:49.866579745-07:00   

                        

In [21]:
'''
Read IMU data from Ouster
'''

acc_times = []
accx = []
accy = []
accz = []

gyrotimes = []
gyrox = []
gyroy = []
gyroz = []

if(IMU == IMUTypes.OUSTER_IMU):
    # Read IMU data
    with open(lidar_json, 'r') as f:
        metadata = client.SensorInfo(f.read())

    source = pcap.Pcap(lidar_pcap, metadata)

    packet_format = client.PacketFormat(metadata)
    for packet in source:
        if isinstance(packet, client.ImuPacket):
            # and access ImuPacket content
            acc_times.append(packet_format.imu_accel_ts(packet.buf))
            accx.append(packet_format.imu_la_x(packet.buf))
            accy.append(packet_format.imu_la_y(packet.buf))
            accz.append(packet_format.imu_la_z(packet.buf))

            gyrotimes.append(packet_format.imu_gyro_ts(packet.buf))
            gyrox.append(packet_format.imu_av_x(packet.buf))
            gyroy.append(packet_format.imu_av_y(packet.buf))
            gyroz.append(packet_format.imu_av_z(packet.buf))

    acc_frame = pd.DataFrame()
    acc_frame['Datetime'] = pd.to_datetime(acc_times)
    acc_frame['Datetime'] = acc_frame['Datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Los_Angeles')
    acc_frame['Acc_x'] = accx
    acc_frame['Acc_y'] = accy
    acc_frame['Acc_z'] = accz

    gyro_frame = pd.DataFrame()
    gyro_frame['Datetime'] = pd.to_datetime(gyrotimes)
    gyro_frame['Datetime'] = gyro_frame['Datetime'].dt.tz_localize('GMT').dt.tz_convert('America/Los_Angeles')
    gyro_frame['Gyro_x'] = gyrox
    gyro_frame['Gyro_y'] = gyroy
    gyro_frame['Gyro_z'] = gyroz

    imu_frame = pd.merge_asof(acc_frame,gyro_frame,on='Datetime')
    imu_frame = imu_frame.dropna()

    print(imu_frame)

[2024-09-23 10:34:08.972] [ouster::sensor] [info] parsing non-legacy metadata format
                                 Datetime     Acc_x     Acc_y     Acc_z  \
1     2024-09-09 19:21:49.474194285-07:00 -0.013672  0.024414  0.970947   
2     2024-09-09 19:21:49.484194224-07:00  0.011963  0.015381  0.975098   
3     2024-09-09 19:21:49.494194065-07:00  0.010742  0.009277  0.985840   
4     2024-09-09 19:21:49.504193971-07:00 -0.014160  0.018555  0.983887   
5     2024-09-09 19:21:49.514193889-07:00 -0.009033  0.016846  0.978760   
...                                   ...       ...       ...       ...   
45247 2024-09-09 19:29:21.976853423-07:00 -0.007812 -0.020996  0.972900   
45248 2024-09-09 19:29:21.986853521-07:00 -0.002441 -0.026367  0.986084   
45249 2024-09-09 19:29:21.996853609-07:00 -0.001709 -0.023926  0.978516   
45250 2024-09-09 19:29:22.006853703-07:00 -0.010254 -0.015869  0.973877   
45251 2024-09-09 19:29:22.016853827-07:00 -0.011230 -0.026367  0.965088   

         Gyro_

In [22]:
"""
Parse gps data marge with perception frame data
Written for .UBX file generated by u-center for u-blox7 USB GPS
"""
# Empty dataframe for gps data
gpsframe = pd.DataFrame()

# Data lists
times = []
latitudes = []
longitudes = []
altitudes = []
altunits = []
alttimes = []

# Parse GPS messages from .ubx file
for mode in (GET, SET, POLL):
    msgcount = 0
    ubr = None
    
    # Open file as U-Center .ubx file
    if(gps_file.endswith('.ubx')):
        print("Reading gps as U-Center output")
        with open(gps_file, "rb") as stream:
            # Create reader
            ubr = UBXReader(stream, quitonerror=ERR_IGNORE, parsing=True,msgmode=mode)

        # Iterate through messages parsed by reader
        for _, parsed in ubr:
            # If message is valid
            if parsed is not None:
                msgcount += 1

                # Grab latitude & longitude if present
                if(hasattr(parsed, 'lat')):
                    times.append(pd.to_datetime(date + str(' ' + parsed.time.strftime('%H:%M:%S'))))
                    latitudes.append(parsed.lat)
                    longitudes.append(parsed.lon)
                # Grab altitude if present
                if(hasattr(parsed,'alt')):
                    altitudes.append(parsed.alt)
                    altunits.append(parsed.altUnit)
                    alttimes.append(pd.to_datetime(date + str(' ' + parsed.time.strftime('%H:%M:%S'))))
            
    # Open file as pygpsclient .log file
    elif(gps_file.endswith('.log')):
        print("Reading gps as pygpsclient binary output")
        with open(gps_file, 'rb') as stream:
            # Create reader
            ubr = UBXReader(stream, quitonerror=ERR_IGNORE, parsing=True, protfilter=UBX_PROTOCOL|NMEA_PROTOCOL|RTCM3_PROTOCOL)

            # Iterate through messages parsed by reader
            for _, parsed in ubr:
                # If message is valid
                if parsed is not None:
                    msgcount += 1

                    # Grab latitude & longitude if present
                    if(hasattr(parsed, 'lat')):
                        time = pd.to_datetime(parsed.time.strftime('%H:%M:%S')) - pd.DateOffset(hours=7)
                        time = time.replace(year=date.year,month=date.month,day=date.day)
                        times.append(time)
                        latitudes.append(parsed.lat)
                        longitudes.append(parsed.lon)
                    # Grab altitude if present
                    if(hasattr(parsed,'alt')):
                        altitudes.append(parsed.alt)
                        altunits.append(parsed.altUnit)
                        time = pd.to_datetime(parsed.time.strftime('%H:%M:%S')) - pd.DateOffset(hours=7)
                        time = time.replace(year=date.year,month=date.month,day=date.day)
                        alttimes.append(time)

    # Print number of messages parsed
    print(f'\n{msgcount} {("GET","SET","POLL")[mode]} messages parsed\n\n')

# Build gps dataframe
gpsframe['Datetime'] = times
gpsframe['Latitude'] = latitudes
gpsframe['Longitude'] = longitudes
gpsframe['Datetime'] = gpsframe['Datetime'].dt.tz_localize('America/Los_Angeles')

# Build separate altitude dataframe since sampling rates are different
altframe = pd.DataFrame()
altframe['Altitude'] = altitudes
altframe['Alt Unit'] = altunits
altframe['Datetime'] = alttimes
altframe['Datetime'] = altframe['Datetime'].dt.tz_localize('America/Los_Angeles')

# Sort data by time
gpsframe = gpsframe.sort_values(by='Datetime')
altframe = altframe.sort_values(by='Datetime')

# Merge GPS and nearest Altitude into GPS dataframe. 0 order hold altitude
gpsframe = pd.merge_asof(gpsframe,altframe,on='Datetime',suffixes=('_gps','_alt'))
gpsframe.replace('', np.nan, inplace = True)
gpsframe = gpsframe.dropna(axis='rows')
gpsframe.to_csv('gpsdata.csv')
print(gpsframe.head())

Reading gps as pygpsclient binary output

3554 GET messages parsed


Reading gps as pygpsclient binary output

3554 SET messages parsed


Reading gps as pygpsclient binary output

3554 POLL messages parsed


                     Datetime   Latitude   Longitude  Altitude Alt Unit
162 2024-09-09 19:21:32-07:00  48.775236 -122.457057      56.1        M
163 2024-09-09 19:21:32-07:00  48.775236 -122.457057      56.1        M
164 2024-09-09 19:21:32-07:00  48.775236 -122.457057      56.1        M
165 2024-09-09 19:21:32-07:00  48.775236 -122.457057      56.1        M
166 2024-09-09 19:21:32-07:00  48.775236 -122.457057      56.1        M


/tmp/ipykernel_5426/1476277003.py:95: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  gpsframe.replace('', np.nan, inplace = True)


In [23]:
"""
Read IMU from Realsense
"""

#TODO:
#1. Change backend timestamp to global timestamp for accurate time delay correction

if (IMU == IMUTypes.REALSENSE_IMU):
    # open IMU file
    rawimu = open(cam_frame_path + imu_file,'r')

    # Empty lists for dataframe
    gyro_data = []
    accel_data = []

    # Remove leading empty lines written by realsense tool
    line = rawimu.readline()
    line = rawimu.readline()    
    line = rawimu.readline()

    # Iterate through gyroscope data
    while(line != '\n'):
        gyro_data.append(line.replace('\n','').split(','))
        line = rawimu.readline()
        print(line)

    # Remove leading empty lines
    line = rawimu.readline()
    line = rawimu.readline()

    # Iterate through accelerometer data
    while(line):
        accel_data.append(line.replace('\n','').split(','))
        line = rawimu.readline()
        print(line)

    # Build pandas dataframe of IMU data
    gyroframe = pd.DataFrame(gyro_data[1:],columns=gyro_data[0])
    gyroframe['Backend Timestamp(ms)'] = gyroframe['Backend Timestamp(ms)'].astype(np.int64).apply(lambda x: pd.to_datetime(x,unit='ms'))
    imuframe = pd.DataFrame(accel_data[1:],columns=accel_data[0])
    imuframe['Backend Timestamp(ms)'] = imuframe['Backend Timestamp(ms)'].astype(np.int64).apply(lambda x: pd.to_datetime(x,unit='ms'))

    # Merge by backend timestamp
    imuframe = pd.merge_asof(imuframe,gyroframe,on='Backend Timestamp(ms)',suffixes=('_Accel','_Gyro'))
    imuframe = imuframe.rename(columns={'Backend Timestamp(ms)':'Datetime'})
    print(imuframe.head())
else:
    print('Skipping realsense IMU')

Skipping realsense IMU


In [24]:
"""
Merge closest lidar and camera frames to single dataframe
Add GPS data
Add IMU data
"""

# Print number of frames to be combined
print("Cam frames: " + str(len(cam_dataframe.index)))
print("Lidar frames: " + str(len(lidar_dataframe.index)))

# Left hand SQL style join on lidar and camera frames
frame_data = pd.DataFrame()
frame_data = pd.merge_asof(lidar_dataframe.sort_values('Datetime'),cam_dataframe.sort_values('Datetime'),on='Datetime')

# Print lone lidar frames
print("Lidar frames with no matching camera: " + str(frame_data.index.size - frame_data.dropna().index.size))
frame_data = frame_data.dropna().reset_index(drop=True)

# Calculate time between camera and lidar timestamps
#frame_data['fused_frame_deltas'] = (frame_data['Datetime'] - frame_data['Datetime_y']).dt.total_seconds()

# Match with closest gps
frame_data = pd.merge_asof(frame_data,gpsframe,on='Datetime')

# If we are saving IMU data from the realsense camera
if(IMU == IMUTypes.REALSENSE_IMU):
    # Match with closest IMU
    frame_data = pd.merge_asof(frame_data,imuframe,on='Datetime')
elif(IMU == IMUTypes.OUSTER_IMU):
    frame_data = pd.merge_asof(frame_data,imu_frame,on='Datetime')

# Save final dataframe to csv
frame_data.to_csv(f'framedata_{date}.csv')
print(frame_data.head())

Cam frames: 13599
Lidar frames: 4527
Lidar frames with no matching camera: 2
   lidar_id                           Timestart  \
0      2225 2024-09-09 19:21:49.466755093-07:00   
1      2226 2024-09-09 19:21:49.566716380-07:00   
2      2227 2024-09-09 19:21:49.666685136-07:00   
3      2228 2024-09-09 19:21:49.766677044-07:00   
4      2229 2024-09-09 19:21:49.866682603-07:00   

                              Timeend  \
0 2024-09-09 19:21:49.566618911-07:00   
1 2024-09-09 19:21:49.666578590-07:00   
2 2024-09-09 19:21:49.766573072-07:00   
3 2024-09-09 19:21:49.866579745-07:00   
4 2024-09-09 19:21:49.966575696-07:00   

                                          lidar_path  \
0  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
1  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
2  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
3  /home/khanj/Dev/PerceptionTools/DriveData/2024...   
4  /home/khanj/Dev/PerceptionTools/DriveData/2024...   

                             Dat

In [25]:
"""
Move camera and lidar to final dataset directory and rename based on index
"""

# Create export directory if does not exist
if (not os.path.exists(dest_path)):
    os.makedirs(dest_path)
if (not os.path.exists(lidar_dest)):
    os.makedirs(lidar_dest)
    os.makedirs(lidar_dest + 'data/')
if (not os.path.exists(cam_dest)):
    os.makedirs(cam_dest)
    os.makedirs(cam_dest + 'data/')
if (not os.path.exists(gps_dest)):
    os.makedirs(gps_dest)
if (not os.path.exists(gps_dest + 'data/')):
    os.makedirs(gps_dest + 'data/')

# Write frame timestamps to pointcloud/Data/
timepath = f'{path}pointcloud/timestamps_start.txt'
if(os.path.exists(timepath) == False):
    with open(timepath, 'w') as f:
        f.write(frame_data['Timestart'].to_string(header=False, index=False))

timepath = f'{path}pointcloud/timestamps_end.txt'
if(os.path.exists(timepath) == False):
    with open(timepath, 'w') as f:
        f.write(frame_data['Timeend'].to_string(header=False, index=False))

timepath = f'{path}pointcloud/timestamps.txt'
if(os.path.exists(timepath) == False):
    with open(timepath,'w') as f:
        f.write(frame_data['Datetime'].to_string(header=False, index=False))
lidar_dataframe.head()

# Write camera timestamps
with open(f'{cam_dest}timestamps.txt','w') as f:
    f.write(pd.to_datetime(frame_data['Epoch'], unit='ms').to_string(header=False, index=False))

# Move matching frames to dataset directory based on index in matched dataframe
for index, row in frame_data.iterrows():    
    shutil.copy(row['lidar_path'], lidar_dest + 'data/' + '{index:010d}'.format(index = index) + '.' + extension)
    shutil.copy(row['camera_path'], cam_dest + 'data/' + '{index:010d}'.format(index = index) + '.png')
    #gpspath = gps_dest + 'data/' + f'{index:010d}'.format(index = index) + '.txt'
    #f = open(gpspath,'x')
    #f.write(str(row['Latitude']) + ' ' + str(row['Longitude']))
    #f.close()

# Copy lidar timestamps to export
shutil.copy(f'{path}pointcloud/timestamps_end.txt', lidar_dest)
shutil.copy(f'{path}pointcloud/timestamps_start.txt', lidar_dest)
shutil.copy(f'{path}pointcloud/timestamps.txt', lidar_dest)

'/home/khanj/Dev/PerceptionTools/DriveData/2024-09-09/Export/pointcloud/timestamps.txt'

In [26]:
'''
Move IMU and GPS data to dataset directory
'''

motion_to_save = ['Latitude', 'Longitude', 'Altitude', 'Acc_x', 'Acc_y', 'Acc_z', 'Gyro_x', 'Gyro_y', 'Gyro_z']

# Write navigation data format to readable file
f = open(f'{gps_dest}dataformat.txt','w')
for field in motion_to_save:
    f.write(field + '\n')
f.close()

# Write GPS data to Export/navigation
for index, row in frame_data.iterrows():
    with open(f'{gps_dest}/data/{index:010d}.txt','w') as f:
        for field in motion_to_save:
            f.write(str(row[field]) + '\n')
